In [2]:
!pip install langchain openai arxiv --quiet


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
import arxiv
from langchain.llms import OpenAI
from langchain.agents import initialize_agent, Tool
from langchain.agents.agent_types import AgentType

def search_arxiv(query: str, max_results: int = 3) -> str:
    results = arxiv.Search(query=query, max_results=max_results).results()
    output = []
    for r in results:
        output.append(f"""
📘 **Title**: {r.title}
👨‍🔬 **Authors**: {', '.join([a.name for a in r.authors])}
📅 **Published**: {r.published.date()}
📄 **Summary**: {r.summary.strip()}
🔗 **URL**: {r.entry_id}
""")
    return "\n".join(output)

    
arxiv_tool = Tool(
    name="ArxivSearch",
    func=lambda q: search_arxiv(q),
    description="Ищет статьи на arXiv.org по заданному научному запросу"
)

# Инициализируем LLM и агента
llm = OpenAI(temperature=0)

agent = initialize_agent(
    tools=[arxiv_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# 🔍 Пример вызова
response = agent.run("Найди и проанализируй последние статьи на тему diffusion models в 2024 году")
print(response)
